# 📝 LLM Classification Fine-tuning (RoBERTa Transformer)

This notebook demonstrates fine-tuning of a RoBERTa-based transformer model to classify preferred responses between pairs of chatbot-generated interactions. It's part of the [LLM Classification Fine-tuning Kaggle competition](https://www.kaggle.com/competitions/llm-classification-finetuning).

**Author**: Chetas Srinivas  
**Date**: March 2025


## ⚙️ Setup & Installation

Install all necessary dependencies including Hugging Face Transformers, datasets, WandB, and other key libraries.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)
import torch
import numpy as np
import wandb

# Initialize wandb (it will prompt for API key)
wandb.login()

## 📂 Data Loading and Exploration

Loading and exploring datasets provided by the competition, which include training and testing data.


In [ ]:
# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

## 🛠️ Data Preprocessing

Prepare textual data for classification tasks by combining prompt and responses. Convert multi-class labels to a single integer class for classification.


In [ ]:
# Prepare data function
def prepare_data(df):
    texts = df['prompt'] + ' [SEP] ' + df['response_a'] + ' [SEP] ' + df['response_b']
    labels = df[['winner_model_a', 'winner_model_b', 'winner_tie']].values.argmax(axis=1)
    return texts.tolist(), labels

train_texts, train_labels = prepare_data(train_df)

## 🧪 Train-Validation Split

Splitting the data into training and validation sets to evaluate model performance and avoid overfitting.


In [ ]:
# Train-validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.1, random_state=42
)

## 🤖 Model and Tokenizer Setup

Using a pre-trained RoBERTa model and tokenizer from Hugging Face for sequence classification.


In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

## 📝 Tokenization

Convert textual data into tokenized sequences suitable for transformer input.


In [ ]:
# Tokenize data with optimized token length
def tokenize(batch):
    return tokenizer(batch, padding=True, truncation=True, max_length=256)

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)

## 📦 PyTorch Dataset Class

Custom dataset class to handle tokenized data efficiently during training and evaluation.


In [ ]:
# Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

## 🚀 Model Training

Training the RoBERTa model with Hugging Face's Trainer API. The training process is logged and visualized using **Weights & Biases (wandb)**.


In [ ]:
# Training arguments (wandb integration)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    report_to="wandb",  # Enables wandb logging
    run_name="llm_classification_finetune",  # Name your wandb run clearly
)

## 📊 Model Evaluation and Predictions

Evaluating the trained model on test data to generate predictions for the competition submission.


In [ ]:
# Metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {'accuracy': (preds == p.label_ids).mean()}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

> **⚠️ Note:**  
> Training was initially executed on Google Colab, and outputs have been preserved here for quick reference and demonstration purposes.


In [ ]:
# Start training
trainer.train()

In [ ]:
# Prepare test data for predictions
test_texts = test_df['prompt'] + ' [SEP] ' + test_df['response_a'] + ' [SEP] ' + test_df['response_b']
test_encodings = tokenize(test_texts.tolist())
test_dataset = Dataset(test_encodings, [0]*len(test_df))

# Predictions
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)

## ✅ Submission File

Generate the final submission file (`submission.csv`) in the format required by Kaggle.


In [ ]:
# Submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'winner_model_a': (preds == 0).astype(int),
    'winner_model_b': (preds == 1).astype(int),
    'winner_tie': (preds == 2).astype(int)
})

submission.to_csv('submission.csv', index=False)

## 🎯 Conclusion & Future Improvements

**Key insights from the current run**:
- Initial accuracy was ~37%. Indicates room for improvement through more extensive hyperparameter tuning and longer training.

**Possible future improvements**:
- Use more advanced models (e.g., DeBERTa).
- Increase epochs and fine-tune hyperparameters.
- Implement hyperparameter optimization techniques.

**Explore further**:
- [wandb Dashboard](https://wandb.ai/chetas0131-sacramento-state/huggingface/runs/83onnn68?nw=nwuserchetas0131)
- [GitHub Repository](https://github.com/Chetas99/LLM_fine_tuning/tree/main)
